In [1]:
# Setup
import pymongo
import pandas as pd
import numpy as np
import ast

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['food_analysis']
urls = db['urls']
recipes = db['recipes']

# Parser Class
class IngredientParser:
    """
    A Parser for processing ingredients
    """
    def __init__(self):
        """
        No params needed, intialization setsup many of the variables needed to run the main function
        .Parse(). This class is only half-baked, it contains a crude and static ingredient reference
        but would need to be widely expanded for general use.
        """
        self.units = ["cup", "c.", "g", "gram", 'lb', 'teaspoon', "tsp", "tbsp", "oz", 'tablespoon', 'container', 'packet',
                 'bag',
                 "stick",
                 'quart', 'pound', 'can', 'bottle', 'pint', 'package', 'ounce', 'jars', 'heads', 'gallons', 'drops',
                 "drop",
                 'envelope', 'bar', 'box', 'pinch', 'dash', 'bunch', 'recipe', 'layer', 'slice', 'link', 'bulb',
                 'stalk',
                 'square', 'sprig',
                 'fillet', 'piece', 'leg', 'thigh', 'cube', 'granule', 'strip', 'tray', 'leave', 'loaves', 'halves',
                 'jar']

        self.descriptors = ['beaten', 'chopped', 'cold', 'diced', 'packed', 'hot', 'large', 'melted', 'mini',
               'miniature', 'packed', 'room temperature', 'sifted', 'softened', 'warm', "drained", "rinsed", "optional",
               "mashed", "ripe", "medium", "ground", "pitted", "minced"]

        self.ir = {
                  "flour":{
           "types":['all purpose', "white", "brown", "spelt", "oat", "millet",
                 "whole wheat", "almond", "tapioca", "sorghum", "rice", "coconut", "garfava"
                 "barley", "cake", "bread", "self-rising", "soy", "pastry"]
                  },
                  "brown sugar":{
                      "types":["dark", "light"],
                      "aka":["brown-sugar"]
                  },
                  "sugar":{
                      "types":["granulated", "white", "substitute", "splenda", "cane"],
                      "aka":["splenda"],
                      "stopwords":["free"]
                  },
                  "butter":{
                      "types":["salted", "unsalted"],
                      "stopwords":["almond", "peanut", "scotch"]
                  },
                  "water":{
                      "types":[]
                  },
                  "baking powder":{
                      "types":[],
                      "aka":["baking-powder"]
                  },
                  "salt":{
                      "types":["kosher", "coarse", "sea", "popcorn", "fine", "himalayan", "table"],
                      "aka":["sea-salt"]
                  },
                  "chocolate chip":{
                      "types":["semi sweet", "milk", "dark", "white", "bittersweet"],
                      "aka":["chocolate chips"]
                  },
                  "chocolate chunk":{
                      "types":["semi sweet", "milk", "dark", "white", "bittersweet"],
                      "aka":["chocolate chunks", "chocolate morsels", "chocolate morsel"]
                  },
                  "baking soda":{
                      "types":[],
                      "aka":["baking-soda"]
                  },
                  "oat":{
                      "types":["old fashioned", "rolled", "quick-cooking"],
                      "aka":["rolled-oats"],
                      "stopwords":["milk", 'flour']
                  },
                  "vanilla extract":{
                      "types":['madagascar'],
                      "aka":["vanilla"]
                  },
                  "shortening":{
                      "types":['vegatable']
                  },
                  "walnut":{
                      "types":["roasted", "toasted"],
                      "aka":["walnuts"],
                      "stopwords":[" nuts"]
                  },
                  "pecan":{
                     "types":["roasted", "toasted"],
                     "aka":["pecans" ]
                  },
                  "oil":{
                      "types":["olive", "vegetable", "avocado", "canola", "virgin", "coconut"]
                  },
                  "coconut":{
                      "types":["flaked", "unsweetened"],
                      "stopwords":["milk", "pudding"]
                  },
                  "chickpeas":{
                      "types":[],
                      "aka":["garbonzo beans", "ghana"]
                  },
                  "cooking spray":{
                      "types":["nonstick"],
                      "aka":["pam"]
                  },
                  "milk":{
                      "types":["whole", "2%", "skim", "dairy free", "coconut", "almond", "oat", "soy"]
                  },
                  "sour cream":{
                      "types":["fat free", "dairy free"]
                  },
                  "banana":{
                      "types":["cavendish"],
                      "stopwords":["extract"]
                  },
                  "banana extract":{
                      "types":[]
                  },
                  "cornstarch":{
                      "types":[]
                  },
                  "cocoa powder":{
                      "types":[]
                  },
                  "nuts":{
                      "types":["walnut", "pecan", "peanut"]
                  },
                  "peanut":{
                      "types":[],
                      "stopwords":[" nuts", "butter"]
                  },
                  "honey":{
                      "types":[]
                  },
                  "baking mix":{
                      "types":[]
                  },
                  "cherry":{
                      "types":[],
                      "aka":["cherries"]
                  },
                  "almond extract":{
                      "types":[]
                  },
                  "almond":{
                      "types":[],
                      "stopwords":["milk", "extract"]
                  },
                  "pudding":{
                      "types":["coconut cream", "butterscotch"]
                  },
                  "heavy cream":{
                      "types":[]
                  },
                  "pretzels":{
                      "types":[]
                  },
                  "date":{
                      "types":["medjool"]
                  },
                  "flax seeds":{
                      "types":[]
                  },
                  "quinoa":{
                      "types":[]
                  },
                  "applesauce":{
                      "types":[]
                  },
                  "xanthan gum":{
                      "types":[]
                  },
                  "cinnamon":{
                      "types":["ground", "saigon"]
                  },
                  "instant coffee":{
                      "types":[]
                  },
                  "peanut butter":{
                      "types":[]
                  },
                  "ginger":{
                      "types":[]
                  },
                  "molasses":{
                      "types":["unsulphured", "un-sulphured", "blackstrap"]
                  },
                  "cloves":{
                      "types":[],
                      "aka":['clove']
                  },
                  "allspice":{
                      "types":[],
                      "aka":["all spice"]
                  },
                  "black pepper":{
                      "types":["ground pepper"]
                  },
                  "strawberry":{
                      "types":[],
                      "aka":['strawberries']
                  },
                  "instant espresso":{
                      "types":[]
                  },
                  "pumpkin pie spice":{
                      "types":[]
                  },
                  "apple pie spice":{
                      "types":[]
                  },
                  "orange peel":{
                      "types":[]
                  },
                  "margarine":{
                      "types":[]
                  },
                  "chocolate":{
                      "types":[]
                  },
                  "egg noodle":{
                      "types":[]
                  },
                  "onion":{
                      "types":["yellow", "spanish", "sweet", "red"]
                  },
                  "ground beef":{
                      "types":[]
                  },
                  "mushroom":{
                      "types":[],
                      "stopwords":['cream of']
                  },
                  "cream of chicken soup":{
                      "types":['undiluted']
                  },
                  "dried parsley":{
                      "types":[]
                  },
                  "garlic":{
                      "types":["clove"]
                  },
                  "cream of mushroom soup":{
                      "types":[]
                  }

                }

    def Parse(self, ing):
        """
        Core function for processing an ingredient
        :param ing:
        :return:
        """

        # Output schema for parsing
        ing_dict = {"ingredient_string": ing.lower(),
                    "quant": None,
                    "unit": None,
                    "item": None,
                    "type": None,
                    "descriptors": None}
        split_ing = ['None', 'None']

        # Unit Parsing
        for unit in self.units:
            if " " + unit + "s " in ing:
                ing_dict["unit"] = unit
                split_ing = ing.lower().split(" " + unit + "s ")
                break
            elif " " + unit + "es " in ing:
                ing_dict["unit"] = unit
                split_ing = ing.lower().split(" " + unit + "es ")
                break
            elif " " + unit + " " in ing:
                ing_dict["unit"] = unit
                split_ing = ing.lower().split(" " + unit + " ")
                break
            elif " " + unit +"." in ing:
                ing_dict["unit"] = unit
                split_ing = ing.lower().split(" " + unit +".")
                break
            else:
                split_ing = [ing, ing]
                continue
        ing_dict["quant"] = self.parse_quant(split_ing[0])

        # ING Parsing
        ing_dict["item"] = self.parse_ing_item(split_ing[1])
        ing_dict["type"] = self.parse_ing_types(split_ing[1], ing_dict["item"])
        ing_dict["descriptors"] = self.parse_ing_descriptors(split_ing[1])

        return ing_dict

    def parse_quant(self, quant):
        """
        Parse out a numerical quantity from fractional and other representations
        :param quant: str containing quantity to conver
        :return: float of quantity
        """
        if quant is None:
            return 0

        fractions = {"↉": "0", "⅒": "1/10", "⅑": "1/9", "⅛": "1/8",
                     "⅐": "1/7", "⅙": "1/6", "⅕": "1/5", "¼": "1/4",
                     "⅓": "1/3", "½": "1/2", "⅖": "2/3", "⅔": "2/3",
                     "⅜": "3/8", "⅗": "3/5", "¾": "3/4", "⅘": "4/5",
                     "⅝": "5/8", "⅚": "5/6", "⅞": "7/8"}

        new_quant = quant.replace("⁄", "/")
        for frac in fractions:
            if frac in quant:
                new_quant.replace(frac, fractions[frac])

        quant_num = 0
        for num in new_quant.split(" "):
            try:
                if "/" in num:
                    try:
                        quant_num += ast.literal_eval(num.split("/")[0]) / ast.literal_eval(num.split("/")[1])
                    except:
                        quant_num += 0
                elif ("-" in num) | ("to" in num):
                    break
                else:
                    quant_num += ast.literal_eval(num)
            except:
                quant_num += 0

        return quant_num

    def parse_ing_descriptors(self, ing_string):
        """
        Loop through possible descriptors for an ingredient and pull them out into a list
        :param ing_string:
        :param descriptors:
        :return:
        """

        descriptors_found = []
        for desc in self.descriptors:

            if desc in ing_string:
                descriptors_found.append(desc)
            elif " " in desc:
                if "-".join(desc.split(" ")) in ing_string:
                    descriptors_found.append(desc)

        return descriptors_found

    def parse_ing_types(self, ing_string, item):
        """
        Parse out the ingredient types for an ingredient string using regex
        :param ing_string: str of ingredient for a recipes ingredient list
        :param item: the item in ir to which the ingredient belongs
        :return: List of types the ingredient matches
        """
        found_types = []
        if item not in self.ir.keys():
            return found_types

        for ing_type in self.ir[item]["types"]:

            if ing_type in ing_string:
                found_types.append(ing_type)
            elif " " in ing_type:
                if "-".join(ing_type.split(" ")) in ing_string:
                    found_types.append(ing_type)

        return found_types

    def parse_ing_item(self, ing_string):
        """
        Determine which item the ingredient is using ir
        :param ing_string: ingredient string to look for
        :return: the item from ir the ingredient corresponds to
        """
        for item in self.ir:

            # Item name
            if item in ing_string:
                good_match = True

                # Check if stopwords
                if "stopwords" in self.ir[item]:
                    for sw in self.ir[item]["stopwords"]:
                        if sw in ing_string:
                            good_match = False

                if good_match:
                    return item
                else:
                    continue

            elif "aka" in self.ir[item].keys():
                for name in self.ir[item]["aka"]:
                    if name in ing_string:
                        return item

In [10]:
# Get recipe list
rec_name = 'banh mi'

In [11]:
query = { "title": { "$regex": rec_name, "$options" :'i' }}
df = pd.DataFrame(list(recipes.find(query)))
df['l_ingredients'] = df.ingredients.astype(str)
df['l_ingredients'] = df['l_ingredients'].str.lower()

In [12]:
def query_ingredient_type(df, col, q):
    
    print(f"{len(df[df[col].str.contains(q)])} / {len(df)}")
    print(f"{len(df[df[col].str.contains(q)]) / len(df)}")

In [28]:
query_ingredient_type(df, 'l_ingredients', 'pickled')

14 / 31
0.45161290322580644


In [29]:
df[~df['l_ingredients'].str.contains('pickled')]

,_id,title,total_time,yields,ingredients,instructions,image,rating,author,reviews,...,cookTime,totalTime,datePublished,recipeYield,recipeCategory,cookingMethod,recipeCuisine,review_count,keywords,l_ingredients
0,5f2209a561c02dddcf3f7b5d,Pork Meatball Banh Mi,0.0,4 item(s),"[2/3 cup mayonnaise, 2 green onions, finely ch...",b'Stir all ingredients in small bowl. Season w...,https://assets.epicurious.com/photos/54b3048e1...,4.000000,None,[{'review_text': 'Fantastic meal! I couldn't f...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['2/3 cup mayonnaise', '2 green onions, finely..."
1,5f24365ce35a40c70b9cf21a,Breakfast Banh Mi Sandwich with Eggs and Sausage,0.0,4 item(s),[2 teaspoons minced scallion (white part only)...,"b'In a large bowl, combine all of the sausage ...",https://assets.epicurious.com/photos/54b944d47...,4.000000,None,[{'review_text': 'This is a good breakfast san...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['2 teaspoons minced scallion (white part only...
4,5f283dce962730a7ad39c5c3,Cauliflower Banh Mi Sandwich,30.0,2 serving(s),[2/3 cup rice wine vinegar ((or sub apple cide...,b'Preheat oven to 450 degrees F (232 C).\nPrep...,https://minimalistbaker.com/wp-content/uploads...,4.980000,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['2/3 cup rice wine vinegar ((or sub apple cid...
5,5f28ad95962730a7ad39fac3,"Vietnamese ""Banh Mi"" Chicken Burger",0.0,4 item(s),"[1 small cucumber, thinly sliced, 1 medium car...","b'In a glass bowl, combine cucumber, carrot an...",https://assets.epicurious.com/photos/54ad63726...,3.500000,None,[{'review_text': 'Very tasty. For those put of...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['1 small cucumber, thinly sliced', '1 medium ..."
6,5f28d196962730a7ad3a0b6c,Banh Mi-Style Hot Dogs,45.0,4 serving(s),"[1 cup julienned cucumber, 1 cup julienned car...","b'In a large bowl, toss the cucumber, carrot, ...",https://food.fnr.sndimg.com/content/dam/images...,5.000000,katie-lee,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['1 cup julienned cucumber', '1 cup julienned ..."
7,5f298531553bb6139d033294,Vietnamese Pork Meatball Banh Mi Fried Rice,0.0,,"[1 small daikon radish, peeled and cut into ma...",b'Combine the daikon and carrot on a colander ...,https://assets.epicurious.com/photos/5d7a5275a...,3.000000,None,[{'review_text': 'I didn't bother making meatb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['1 small daikon radish, peeled and cut into m..."
9,5f29a1a3553bb6139d033ffb,Tofu Spring Rolls with Pickled Veggies (Banh M...,45.0,12 serving(s),"[2 cups vegetables ((carrots, daikon or red ra...",b'Preheat oven to 400 degrees F (204 C) and wr...,https://minimalistbaker.com/wp-content/uploads...,4.780000,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['2 cups vegetables ((carrots, daikon or red r..."
10,5f29d6f7553bb6139d03599b,Banh Mi-Style Chicken Sandwich,15.0,4 serving(s),"[4 (8 inch) French baguettes, split, 1⁄4 cup l...","b'To make sandwiches, spread both sides of the...","https://img.sndimg.com/food/image/upload/q_92,...",5.000000,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['4 (8 inch) french baguettes, split', '1⁄4 cu..."
12,5f2ad9c4cd354678e6615d0c,Pork Banh Mi Bowls,60.0,None,"[1 pound ground pork, 2 teaspoons minced garli...",b'',https://www.lecremedelacrumb.com/wp-content/up...,4.500000,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['1 pound ground pork', '2 teaspoons minced ga..."
13,5f2ae1ada081e94c3de8cf21,Vietnamese Chicken Sandwich (Banh Mi),0.0,4 item(s),"[1/2 pound daikon, peeled, 1 carrot, peeled, 1...",b'Preheat oven to 350\xc2\xb0F with rack in mi...,https://assets.epicurious.com/photos/560d77e6f...,3.500000,None,[{'review_text': 'Wonderful. I've made this se...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['1/2 pound daikon, peeled', '1 carrot, peeled..."
